# Introduction

Hi...

### Initialization

In [598]:
# Import visualization libraries
import plotly.graph_objects as go
import plotly.express as px

# Import file management libraries
import csv as csv
from operator import itemgetter

In [599]:
# Initialize data
dataList = []
gpaDataList = []

# Start file operations
with open('records.csv') as records:
    records.__next__() # Skip the header row
    reader = csv.reader(records, delimiter=',') # Read unsorted file
    
    for row in reader:
        row[0] = row[0][2:] # Trim off the 'G-' to allow for sorting
        dataList.append(row)

# Sort dataList function by tutorial group
def sortData(data):
    for row in data:
        row[0] = int(row[0])
    
    # Using itemgetter to retrieve the row by the column index, e.g., Tutorial Group is index 0 - First Column
    sortedData = sorted(data, key=itemgetter(0))
    return sortedData

# Sort dataList by tutorial group and CGPA
def sortCGPA(data):
    for row in data:
        row[0] = int(row[0])
        row[5] = float(row[5])
    
    # Using itemgetter to retrieve the row by the column index,
    # e.g., Tutorial Group is index 0 - First Column
    #       CGPA is index 5 - Sixth Column
    sortedData = sorted(data, key=itemgetter(0, 5))
    return sortedData

In [600]:
# Graph Visualization functions using Plotly
#TODO: Exception Handling
def showCGPAPerGroup():
    cgpa = []
    tgrp = []
    for student in sortCGPA(dataList):
        cgpa.append(student[5])
        tgrp.append(student[0])
        # print(str(student[0]) + ' ' + str(student[5])) # Use for checking sorting

    # Make it into a readable box plot using the python arrays above
    boxTrace = go.Box(
        x = tgrp,
        y = cgpa,
        name = 'CGPA / Tutorial Group'
    )

    # Initialize the graph object with go.Figure()
    fig = go.Figure(boxTrace)
    fig.update_layout(
        title="CGPA / Tutorial Group",
        xaxis_title="Tutorial Group",
        yaxis_title="CGPA") # Name the title of the graph
    fig.show() # Show the graph
    
#showCGPAPerGroup()

In [601]:
#TODO: Exception Handling
def showStudentsPerGroup():
    # Graph on No. of Students Per Tutorial Group
    numPerGrp = {}
    studentList = sortData(dataList)

    for student in studentList:
        if student[0] not in numPerGrp.keys():
            numPerGrp[student[0]] = 0

    for updateStudent in studentList:
        numPerGrp[updateStudent[0]] += 1

    barTrace = go.Bar(
        x = list(numPerGrp.keys()),
        y = list(numPerGrp.values()),
        name = 'Number of Students / Tutorial Group'
    )

    # Initialize the graph object with go.Figure()
    fig = go.Figure(barTrace)
    fig.update_layout(
        title="No. of Students / Tutorial Group",
        xaxis_title="Tutorial Group",
        yaxis_title="No. Of Students") # Name the title of the graph
    fig.show() # Show the graph
    
# showStudentsPerGroup()

In [602]:
#TODO: Exception Handling
def showFacultyPerGroup(tutorialGrpNum):
    # Graph on Faculty Distribution per Tutorial Group
    facPerGrp = {}
    numPerGrp = {}
    studentList = sortData(dataList)

    # Get the number of students per tutorial group
    for student in studentList:
        if student[0] not in numPerGrp.keys():
            numPerGrp[student[0]] = 0
                
        if student[2] not in facPerGrp.keys():
            facPerGrp[student[2]] = 0

    for updateStudent in studentList:
        numPerGrp[updateStudent[0]] += 1

    # Input the tutorial group number to show the distribution for that group
    tgrpToShow = tutorialGrpNum

    # Get the number of students per faculty in a tutorial group
    for i in range(numPerGrp[tgrpToShow] * tgrpToShow - 50,
                numPerGrp[tgrpToShow] * tgrpToShow):
        facPerGrp[studentList[i][2]] += 1 # incremement the facPerGrp dict by referencing the indexed student and their faculty

    barTrace1 = go.Bar(
            x = list(facPerGrp.keys()),
            y = list(facPerGrp.values()),
            name = 'No. of Students / Faculty'
        )

    # Initialize the graph object with go.Figure()
    fig = go.Figure(barTrace1)
    fig.update_layout(
        title="No. of Students / Faculty",
        xaxis_title="Faculty",
        yaxis_title="No. Of Students") # Name the title of the graph
    fig.show() # Show the graph
    
# showFacultyPerGroup(1)

In [ ]:
#TODO: Exception Handling
def showGenderPerGroup():
    # Graph on Gender Count per Tutorial Group 
    tgrp = []
    malePerGrp = {}
    femalePerGrp = {}
    studentList = sortData(dataList)

    for student in studentList:
        if student[0] not in tgrp:
            tgrp.append(student[0])
            malePerGrp[student[0]] = 0
            femalePerGrp[student[0]] = 0
        
    for updateNum in studentList:
        if updateNum[4] == 'Male':
            malePerGrp[updateNum[0]] += 1
        if updateNum[4] == 'Female':
            femalePerGrp[updateNum[0]] += 1

    maleTrace = go.Bar(
        x = tgrp,
        y = list(malePerGrp.values()),
        hovertext='(Tutorial Group, No. Of Males)',
        name = 'Male')

    femaleTrace = go.Bar(
        x = tgrp,
        y = list(femalePerGrp.values()),
        hovertext = '(Tutorial Group, No. Of Females)',
        name = 'Female')

    # Initialize the graph object with go.Figure()
    fig = go.Figure()
    fig.add_trace(maleTrace)
    fig.add_trace(femaleTrace)
    fig.update_layout(
        title = "Gender / Tutorial Group", # Name the title of the graph
        xaxis_title = "Tutorial Group",
        yaxis_title = "No. Of Students")
    fig.show() # Show the graph
    
# showGenderPerGroup()